In [1]:
import copy
import numpy as np
from collections import deque

### Load data

In [3]:
raw_data = []

with open("./data/PortlandCirc.kp", "r") as f:
    for lines in f:
        raw_data.append(lines)

In [4]:
n_line = int(raw_data[0])

if("nl" in raw_data[1]):
    raw_relationship = raw_data[2: 2 + n_line]
else:
    raw_relationship = raw_data[1 + n_line: 1 + n_line * 2]
    
raw_relationship[0: 5]

['00010110110101001\n',
 '00001001001000000\n',
 '00000001001010110\n',
 '10000110110001100\n',
 '01000001001000010\n']

In [5]:
data_raw = []

for lines in raw_relationship:
    lines = lines[: -1]
    data_raw.append([int(nums) for nums in list(lines)])

data_raw[: 5]

[[0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0],
 [1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0]]

### Number of Nodes

In [6]:
nums_component = len(data_raw)

print(nums_component)

17


### Number of Ties

In [7]:
nums_tie = 0

data = copy.deepcopy(data_raw)
length = len(data)

for row in range(length):
    for col in range(length):
        if(data[row][col] == 1):
            nums_tie += 1
            data[col][row] = 0
            
print(nums_tie)

41


### Number of Components

In [8]:
def dfs_double(data, row):
    
    '''
    1. Find all the elements that connects to (1s in the rows)
    2. Find all the elements that is connected to (1s in the cols)
    '''
    
    length = len(data)
    
    for items in range(length):
        if(data[row][items] == 0):
            continue
        data[row][items] = 0
        data = dfs_double(data, row)
    
    for items in range(length):
        if(data[items][row] == 0):
            continue
        data[items][row] = 0    
        data = dfs_double(data, items)
        
    return data

In [9]:
nums_component = 0

data = copy.deepcopy(data_raw)
length = len(data)

#get the elements that is not connected
for i in range(len(data)):
    if(1 not in data[i] and 1 not in np.array(data).T[i]):
        nums_component += 1

#use dfs to get the number of islands
for row in range(length):
    for col in range(length):
        if(data[row][col] == 1):
            nums_component += 1
            data = dfs_double(data, row)
            
            
print(nums_component)

1


### The largest shortest path

In [10]:
# This will take in a node and matrix of connections to get all its neighbour nodes
def getConnections(node,networkMatrix):
    
    # List for connections
    nodeConnections = []
    
    # Getting the maximum number of nodes
    numNodes = networkCon.shape[0]
    
    # Loop through all nodes
    for currNode in range(0,numNodes):
        
        # If the provided connection matrix index has a connection...
        if networkMatrix[node,currNode]:
            
            # ... Append it to the connections list
            nodeConnections.append(currNode)
    
    # Return the list of connections
    return nodeConnections

In [11]:
# Breadth first search stores parent nodes when searching
# This goes from parent nodes array to shortest path
# Takes in the end node plus an array corresponding to parents of all nodes
def decodeNodeParents(endNode,nodeParents):
    
    # List to store shortest path
    shortestPath = []
    
    # -2 represents end node has no parrent meaning no connections, return empty
    if nodeParents[endNode] == -2:
        return shortestPath
    
    # Set the current parent to end nodes parent, put end node in shortest path
    currParent = nodeParents[endNode]
    shortestPath.append(endNode)
    
    # -1 represents start node, loop until current parent is start node
    while currParent != -1:
        
        # Add the current parent to the shortest path list
        shortestPath.append(currParent)
        
        # Update current parent to the parent of the current parent
        currParent = nodeParents[currParent]
    
    # currParent goes from end -> start, reverse to start -> end and return
    return shortestPath[::-1]

In [12]:
# Breadth first search algorithm to find shortest path
# Takes in start node, end node, and the matrix of connections between nodes
def getShortestPath(startNode,endNode,networkCon):
    
    # Lower and upper bounds to check if start and end nodes are in range
    lowBound = 0
    upBound = networkCon.shape[0]
    
    # Check if start node is out of range, return empty list if true
    if (startNode < lowBound or startNode > upBound):
        return []
    
    # Check if end node is out of range, return empty list if true
    if (endNode < lowBound or endNode > upBound):
        return []
    
    # Check if start node and end node are the same, return empty list if true
    if (startNode == endNode):
        return []
    
    # Create queue of nodes for search
    queueNodes = deque()
    queueNodes.append(startNode)
    
    # Create list of visited nodes
    visitedNodes = [startNode]
    
    # Create a list of parent nodes
    # -2 means no connections, whether it physically has no connections or wasn't visited in search
    # -1 means the start node
    # Starts as -2 for all nodes except start node, which will be -1
    nodeParents = np.ones(networkCon.shape[0],dtype=int)*-2
    nodeParents[startNode] = -1
    
    # Used to kick out of while loop when we find the end node
    foundEnd = False
    
    # Loop while there are nodes to check for in our queue
    while len(queueNodes) != 0:
        
        # Get the new node to check
        currNode = queueNodes.popleft()
        
        # Get the new nodes connections
        currNodeCon = getConnections(currNode,networkCon)
        
        # Loop through all the nodes connections...
        for oneNode in currNodeCon:
            
            # ... if we haven't visited the node before...
            if not(oneNode in visitedNodes):
                
                # Set the parent of the new node to the connection node in the node parents array
                nodeParents[oneNode] = currNode
                
                # Set the connection node as visited
                visitedNodes.append(oneNode)
                
                # Add the connection node to the queue
                queueNodes.append(oneNode)
                
                # If the connection node is the end node, break out of overall loop
                if oneNode == endNode:
                    foundEnd = True
                    break
        
        # If end node is found, break out of overall loop
        if foundEnd:
            break
    
    # Decode the shortest path that was found
    shortestPathList = decodeNodeParents(endNode,nodeParents)
    
    # Return the list of the shortest path
    return shortestPathList

In [13]:
# This takes in a matrix of network connections,
# Loops through it
# And finds the largest of shortest paths with breadth first search
def getLargestShortestPath(networkCon):
    
    # Holder for largest number of edges
    largestNumEdges = 0
    
    # Loop through all nodes in the graph
    for startNode in range(0,networkCon.shape[0]):
        # Loop through the rest of the nodes in combinations not visited yet
        for endNode in range(startNode+1,networkCon.shape[0]):
            
            # Get the shortest path between the nodes
            onePath = getShortestPath(startNode,endNode,networkCon)
            
            # If the number of nodes in path is greater than previously found, replace with new number
            if len(onePath)-1 > largestNumEdges:
                largestNumEdges = len(onePath)-1
    
    # Return largest value
    return largestNumEdges

In [16]:
data = copy.deepcopy(data_raw)
networkCon = np.array(data)
largestNumEdges = getLargestShortestPath(networkCon)

print(largestNumEdges)

5


In [18]:

#For test

import pandas as pd

test = pd.DataFrame(data_raw)
test



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,0,0,1,0,1,1,0,1,1,0,1,0,1,0,0,1
1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0
3,1,0,0,0,0,1,1,0,1,1,0,0,0,1,1,0,0
4,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0
5,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
6,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
7,0,1,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0
8,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0
9,1,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0
